# MONTHLY DCA WITH ROUND LOT

In [6]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock

In [6]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock

def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

class DCA(Strategy):
    average_monthly_income_vnd =  1000  # 10.000.000
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0

    def init(self):
        self.amount_to_invest = self.average_monthly_income_vnd * self.investment_percentage 
        self.first_mondays = calculate_first_mondays(self.data.index)

    def next(self):
        today = self.data.index[-1]
        self.data.Close[-1] = self.data.Close[-1]*0.1
        if today in self.first_mondays:
            # self.buy(size=math.floor(self.amount_to_invest / self.data.Close[-1]))
            share_price = self.data.Close[-1]
            shares_to_buy = self.amount_to_invest // (share_price)# 16450 => 16.45 VNSTOCK => 1.6450
            if shares_to_buy >= 100:
                shares_to_buy = (shares_to_buy // 100) * 100
                self.buy(size=shares_to_buy)
                #print('no fund')
                #print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
            else:
                self.fund += self.amount_to_invest
                #print(f'{self.fund} before buy')
                shares_to_buy = self.fund // (share_price)
                if shares_to_buy >= 100:
                    shares_to_buy = (shares_to_buy // 100) * 100
                    self.fund -= shares_to_buy * share_price
                    self.buy(size=shares_to_buy)


def run_backtest(stock_symbol):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol, source='VCI').quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)
    stock_data = stock_data.dropna()
    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]
    #print(trades)
    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    print("-" * 50)

# List of stock symbols
stock_symbols = ['FPT', 'MWG', 'E1VFVN30']  # Add more stock symbols as needed

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol)


2024-10-24 20:08:17,617 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


2024-10-24 20:08:18,593 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for FPT:
Total investment: 5984.200000000001
Current Shares: 1700
Current Equity: 14137.2
RoR: 136.24210420774705
--------------------------------------------------


2024-10-24 20:08:19,409 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Results for MWG:
Total investment: 5823.700000000001
Current Shares: 1500
Current Equity: 6429.000000000001
RoR: 10.393735941068396
--------------------------------------------------
Results for E1VFVN30:
Total investment: 5986.0
Current Shares: 3400
Current Equity: 6694.600000000001
RoR: 11.837621115937209
--------------------------------------------------


## BUY MONTHLY WITH ROUND-LOT

In [8]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import numpy as np
from datetime import datetime
import datetime as dt

def calculate_first_mondays(dates):
        if not isinstance(dates, pd.DatetimeIndex):
            dates = pd.DatetimeIndex(dates)
        dates_series = pd.Series(dates, index=dates)
        mondays = dates_series[dates_series.dt.dayofweek == 0]
        first_mondays = mondays.groupby([mondays.dt.year, mondays.dt.month]).first()
        return set(first_mondays)

class DCA(Strategy):
    average_monthly_income_vnd =  1000  # 10.000.000
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0
    def init(self):
        self.amount_to_invest = self.average_monthly_income_vnd * self.investment_percentage 
        self.first_mondays = calculate_first_mondays(self.data.index)
        # self.day_of_week = self.I(
        #     lambda x: x,
        #     self.data.Close.s.index.dayofweek,
        #     plot=False,
        # )

    def next(self):
        today = self.data.index[-1]
        self.data.Close[-1] = self.data.Close[-1]*0.1
        if today in self.first_mondays:
            # self.buy(size=math.floor(self.amount_to_invest / self.data.Close[-1]))
            share_price = self.data.Close[-1]
            shares_to_buy = self.amount_to_invest // (share_price)# 16450 => 16.45 VNSTOCK => 1.6450
            if shares_to_buy >= 100:
                shares_to_buy = (shares_to_buy // 100) * 100
                self.buy(size=shares_to_buy)
                #print('no fund')
                #print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")
            else:
                self.fund += self.amount_to_invest
                #print(f'{self.fund} before buy')
                shares_to_buy = self.fund // (share_price)
                if shares_to_buy >= 100:
                    shares_to_buy = (shares_to_buy // 100) * 100
                    self.fund -= shares_to_buy * share_price
                    self.buy(size=shares_to_buy)
                    #print(f"Buy by fund executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}(ĐƠN VỊ LÀ 10000)")
                    #print(f'{self.fund} after buy')
def run_backtest(stock_symbol, start_date, end_date):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol, source='VCI').quote.history(start=start_date, end=end_date)
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)
    # Merge USD/VND data
    #stock_data['usd/vnd'] = usd_vnd_data['Close'] / 1000
    #stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']

    # Handle NaN values (choose one method)


    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    #bt.plot(filename=f'{stock_symbol}')
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]
    #print(trades)
    #print(f"Results for {stock_symbol}:")
    #print("Total investment:", total_invested)
    #print("Current Shares:", current_shares)
    #print("Current Equity:", current_equity)
    #print("RoR:", ((current_equity - total_invested) / total_invested)*100)
    #print('profoit/loss: ', current_equity - total_invested)
    #print("-" * 50)
    return {
        'stock_symbol': stock_symbol,
        'total investment': total_invested,
        'current shares': current_shares,
        'current_equity': current_equity,
        'ror': ((current_equity - total_invested) / total_invested) * 100
    }
# usd_vnd_data = pd.read_csv('D:/datcom lab/BACKTESTING LIBRARY LEARNING/VND=XCommon.csv')
# #usd_vnd_data = pd.read_csv(('D:/datcom lab/2nd paper/VND=X_Common.csv'))
# usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
# usd_vnd_data.set_index('Date', inplace=True)
# usd_vnd_data.columns = usd_vnd_data.columns.str.strip()
# List of stock symbols
#stock_symbols = ['SSI', 'EIB', 'HPG']  # Add more stock symbols as needed
stock_symbols = ['FPT', 'MWG', 'E1VFVN30']
date_start = '2019-01-01'
date_end = ['2020-01-01', '2021-01-01', '2022-01-01', '2023-01-01', '2024-01-04']
dict_symbol = {}
for symbol in stock_symbols:
    for date in date_end:
        # Convert date string to datetime object to extract the year
        date_obj = datetime.strptime(date, '%Y-%m-%d')
        key = f'{symbol}_{date_obj.year}'
        # Run the backtest for each stock symbol and date range
        dict_symbol[key] = run_backtest(symbol, start_date=date_start, end_date=date)

dict_symbol

2024-10-11 22:13:34,407 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:36,580 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:39,098 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:41,145 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:43,177 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:44,992 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:47,632 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
2024-10-11 22:13:49,927 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yế

{'FPT_2020': {'stock_symbol': 'FPT',
  'total investment': 1191.6000000000001,
  'current shares': 600,
  'current_equity': 1491.0000000000002,
  'ror': 25.125881168177244},
 'FPT_2021': {'stock_symbol': 'FPT',
  'total investment': 2135.7000000000003,
  'current shares': 1000,
  'current_equity': 3015.0,
  'ror': 41.171512852928764},
 'FPT_2022': {'stock_symbol': 'FPT',
  'total investment': 3454.3,
  'current shares': 1300,
  'current_equity': 7241.0,
  'ror': 109.62278898763857},
 'FPT_2023': {'stock_symbol': 'FPT',
  'total investment': 4582.3,
  'current shares': 1500,
  'current_equity': 8463.0,
  'ror': 84.68891168190645},
 'FPT_2024': {'stock_symbol': 'FPT',
  'total investment': 5984.200000000001,
  'current shares': 1700,
  'current_equity': 14137.2,
  'ror': 136.24210420774705},
 'MWG_2020': {'stock_symbol': 'MWG',
  'total investment': 939.0,
  'current shares': 300,
  'current_equity': 1094.1000000000001,
  'ror': 16.51757188498404},
 'MWG_2021': {'stock_symbol': 'MWG',
  